In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install rouge_score
!pip install nltk
!pip install pythainlp
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=a46bc75b3580326cc4aaef5e27811ae35e2864d73635f89480fa52a1efa27e26
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
sum_dataset = load_dataset("csv", data_files={"train": "/content/drive/MyDrive/thaisum/trainsum_only_news.csv", "validation": "/content/drive/MyDrive/thaisum/validation_set.csv", "test": "/content/drive/MyDrive/thaisum/test_set.csv"})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e42fb7def268192c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
sum_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 128959
    })
    validation: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
})

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "/content/drive/MyDrive/thaisum/Thaisum_model/e4_1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = sum_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/128959 [00:00<?, ? examples/s]

Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 1
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size

args = Seq2SeqTrainingArguments(
    output_dir="/content/e5",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
)

In [ ]:
import numpy as np
from pythainlp.tokenize import sent_tokenize
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip(), engine="crfcut")) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip(), engine="crfcut")) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    sum_dataset["train"].column_names
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.078000,0.388706,19.758600,5.866500,19.721500,19.737000


TrainOutput(global_step=16120, training_loss=2.0780364579361663, metrics={'train_runtime': 9264.7645, 'train_samples_per_second': 13.919, 'train_steps_per_second': 1.74, 'total_flos': 6.816716196421632e+16, 'train_loss': 2.0780364579361663, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/e5_1")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r "/content/thaisum_epoch_1.zip" "/content/Thaisum_model"

  adding: content/Thaisum_model/ (stored 0%)
  adding: content/Thaisum_model/checkpoint-16000/ (stored 0%)
  adding: content/Thaisum_model/checkpoint-16000/trainer_state.json (deflated 42%)
  adding: content/Thaisum_model/checkpoint-16000/tokenizer_config.json (deflated 39%)
  adding: content/Thaisum_model/checkpoint-16000/optimizer.pt (deflated 41%)
  adding: content/Thaisum_model/checkpoint-16000/spiece.model (deflated 46%)
  adding: content/Thaisum_model/checkpoint-16000/config.json (deflated 47%)
  adding: content/Thaisum_model/checkpoint-16000/tokenizer.json (deflated 76%)
  adding: content/Thaisum_model/checkpoint-16000/training_args.bin (deflated 49%)
  adding: content/Thaisum_model/checkpoint-16000/pytorch_model.bin (deflated 21%)
  adding: content/Thaisum_model/checkpoint-16000/special_tokens_map.json (deflated 35%)
  adding: content/Thaisum_model/checkpoint-16000/scheduler.pt (deflated 48%)
  adding: content/Thaisum_model/checkpoint-16000/rng_state.pth (deflated 28%)
  adding

In [ ]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/thaisum/Thaisum_model/e5_1")

In [ ]:
abstract = "หากไม่หลอกตัวเองจนเกินไป จะพบว่าสังคมไทยไม่มีอะไรเหมือนเดิมอีกต่อไปแล้ว ปฏิกิริยาอันหลากหลายที่สะท้อนผ่านความรู้สึกนึกคิดและการแสดงออกของบรรดาผู้คน แสดงนัยยะให้เห็นชัดเจนว่าประเทศไทยได้ก้าวเข้าสู่ยุคเปลี่ยนผ่านครั้งสำคัญจากความขัดแย้งทางการเมืองในสังคมไทยเมื่อห้าปีก่อน หากผู้เกี่ยวข้องต่อการนี้ได้เข้าใจกันบ้างว่ากับบางเรื่องราว ไม่ว่าจะมีเหตุผลอย่างไร ก็ไม่สมควรกระทำแล้ว การรัฐประหารเมื่อวันที่ ๑๙ กันยายน ๒๕๔๙ ก็คงไม่เกิด และเมื่อการยึดอำนาจจากรัฐบาลพรรคไทยรักไทยไม่ได้มีผลเป็นการล้มล้างรัฐบาลอย่างเดียว แต่ทว่ายังเป็นการทำลายอำนาจการตัดสินใจของประชาชนผู้สนับสนุนรัฐบาลด้วย จากการกระทำด้วยวิธีการที่มิชอบดังนี้ จึงถือเป็นการเปิดม่านให้กับการมาเยือนของวิกฤติการเมืองไทยดังที่เห็นในปัจจุบัน ความจริงแล้ว ปัญหาทางการเมืองอาจจะไม่ขยายตัว หากคู่กรณีฝ่ายหนึ่งมีความยับยั้งชั่งใจต่อการใช้อำนาจของตนอยู่บ้าง แต่หลังจากการรัฐประหารสำเร็จลุล่วง เมื่อปฏิบัติการทำลายล้างทางการเมืองต่อคู่กรณีฝ่ายตรงกันข้ามยังคงดำเนินไปอย่างต่อเนื่อง กล่าวคือ พรรคไทยรักไทยได้ถูกยุบไปโดยคำวินิจฉัยของคณะตุลาการรัฐธรรมนูญ หัวหน้าพรรคพลังประชาชนซึ่งดำรงตำแหน่งนายกรัฐมนตรีถูกศาลรัฐธรรมนูญ วินิจฉัยให้พ้นจากตำแหน่งผู้นำรัฐบาล และพรรคพลังประชาชนถูกยุบตามไปโดยคำวินิจฉัยของศาลรัฐธรรมนูญผลจากการที่ถูกกระทำซ้ำซากดังนี้ ทำให้ความอดทนของผู้ที่สนับสนุนพรรคการเมืองทั้งสองพรรคเดินมาถึงจุดสิ้นสุด และนำไปสู่การชุมนุมประท้วงแบบยืดเยื้อตามมาหลายครั้ง ครั้งล่าสุดก็เกิดเหตุการณ์สลายการชุมนุมเมื่อเดือนพฤษภาคม ๒๕๕๓ ซึ่งสร้างความเสียหายให้กับชีวิตและทรัพย์สินของผู้คนอย่างไม่อาจประเมินค่าได้ ทั้งยังสร้างความบาดหมางในสังคมไทยให้ร้าวลึกลงไปแบบยากที่จะเยียวยากระนั้นก็ตาม ไม่ว่าความขัดแย้งจะขยายตัวเท่าใด สิ่งที่ต้องตราไว้ก็คือ ความไม่ลงรอยทางการเมืองดังนี้ย่อมไม่อาจนำพาประเทศเข้าสู่ยุคเปลี่ยนผ่าน ได้ หากผู้ที่มีส่วนเกี่ยวข้องกับการนี้เป็นเพียงคนเฉพาะกลุ่มระหว่าง นักการเมืองด้วยกัน อย่างไรก็ดี เมื่อคู่ขัดแย้งทางการเมืองคราวนี้ประกอบอยู่ทั้งองคมนตรี กองทัพ ตุลาการ พรรคการเมือง และประชาชนซึ่งเป็นผู้สนับสนุนคู่กรณีของแต่ละฝ่าย ความขัดแย้งในสังคมไทยจึงลุกลามบานปลายจนกลายเป็นวิกฤติทางการเมืองอย่างคาดคิดไม่ถึงคงไม่จำเป็นอีกแล้วต่อการสาธยายรายละเอียดถึงบทบาทของคู่ขัดแย้งทางการเมืองในครั้งนี้ เพราะข้อเท็จจริงเท่าที่ปรากฏในทางต่างๆ รวมไปถึงบันทึกของเอกอัครราชทูตสหรัฐอเมริกาประจำประเทศไทยที่มีไปถึงวอชิงตันเกี่ยวกับปัญหาการเมืองไทย ซึ่งถูกนำมาตีแผ่ผ่านวิกิลีกส์ว่ามีบุคคลใดเกี่ยวข้องกับเรื่องใดบ้าง ก็เพียงพอต่อการทำให้สถาบันสำคัญในสังคมไทยตกอยู่ในภาวะแทบจะล้มละลายต่อความน่าเชื่อถือสำหรับผู้คนจำนวนหนึ่งเสียสิ้น และจากความรู้สึกดังนี้ ผู้คนจำนวนไม่น้อยจึงเห็นถึงความจำเป็นของการต้องเปลี่ยนผ่านประเทศไทยเสียทีไม่ใช่หัวใจสำคัญของเรื่องเสียแล้ว หากจะตั้งคำถามอยู่เพียงว่าพรรคการเมืองใดจะเป็นผู้จัดตั้งรัฐบาลและใครจะมาดำรงตำแหน่งนายกรัฐมนตรีเมื่อมีการเลือกตั้ง แต่ทว่าคำถามที่สอดรับกับสถานการณ์ทางข้อเท็จจริงมากที่สุดขณะนี้ย่อมอยู่ที่ว่า จะเปลี่ยนผ่านประเทศไทยอย่างไรให้ก้าวข้ามความขัดแย้งไปยังจุดหมายที่ควรจะ เป็น ตามทำนองคลองธรรมของการปกครองในระบอบประชาธิปไตยอย่างแท้จริงเพื่อให้การเปลี่ยนผ่านประเทศไทยเป็นไปอย่างสันติ ทุกสถาบันสำคัญในสังคมไทยจำต้องพร้อมจะรับฟังคำวิพากษ์วิจารณ์บนพื้นฐานของข้อเท็จจริงว่าตนจะต้องปรับตัวหรือเปลี่ยนแปลงอย่างไร เพื่อมิให้สถาบันของตนกลายเป็นต้นเหตุของวิกฤติทางการเมืองซึ่งทำให้ประเทศไทยต้องตกอยู่ในภาวะที่กลืนไม่เข้าคายไม่ออกเหมือนดังที่เป็นอยู่ในปัจจุบันวันนี้สังคมไทยได้ย่างก้าวเข้าสู่ยุคแห่งการเปลี่ยนผ่านแล้ว และต้องตระหนักว่าการหยุดยั้งความเปลี่ยนแปลงอันเป็นกฎธรรมดาโลกไม่เคยมีผู้ ใดกระทำได้สำเร็จ ไม่ว่าจะโดยอาศัยอำนาจตามกฎหมายหรือใช้แสนยานุภาพทางอาวุธเป็นเครื่องเหนี่ยวรั้ง เมื่อประเทศไทยไม่อาจหวนกลับไปเป็นอย่างเดิมอีกได้ สถาบันการเมืองทั้งหลายจึงต้องหาจุดลงตัวที่เหมาะสมของตนให้ได้เพื่อให้สอดรับกับสภาพแวดล้อมอย่างใหม่ที่ประเทศไทยกำลังจะเปลี่ยนผ่านไปถึงหากสถาบันการเมืองใดยังคิดว่าตนอยู่เหนือกฎแห่งการเปลี่ยนแปลง อนาคตไม่ไกลจากนี้จะเป็นเครื่องชี้วัดให้เห็นว่าสิ่งที่ท่านคิดนั้นผิดหรือถูก"

In [ ]:
input_ids = tokenizer.encode(abstract, return_tensors="pt", add_special_tokens=True)
# generated_ids = loaded_model.generate(input_ids=input_ids,num_beams=5,max_length=50,repetition_penalty=2.5,length_penalty=1,early_stopping=True,num_return_sequences=3)
generated_ids = loaded_model.generate(input_ids=input_ids,num_beams=5,repetition_penalty=2.5,length_penalty=1,early_stopping=True,num_return_sequences=1, max_new_tokens=50)
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print(preds)

['ความขัดแย้งทางการเมืองในสังคมไทยจึงลุกลามบานปลายจนกลายเป็นวิกฤติทางการเมืองอย่างคาดคิดไม่ถึง ']


In [ ]:
!zip -r "/content/thaisum_epoch_1.zip" "/content/Thaisum_model/checkpoint-16000"

  adding: content/Thaisum_model/checkpoint-16000/ (stored 0%)
  adding: content/Thaisum_model/checkpoint-16000/trainer_state.json (deflated 42%)
  adding: content/Thaisum_model/checkpoint-16000/tokenizer_config.json (deflated 39%)
  adding: content/Thaisum_model/checkpoint-16000/optimizer.pt (deflated 41%)
  adding: content/Thaisum_model/checkpoint-16000/spiece.model (deflated 46%)
  adding: content/Thaisum_model/checkpoint-16000/config.json (deflated 47%)
  adding: content/Thaisum_model/checkpoint-16000/tokenizer.json (deflated 76%)
  adding: content/Thaisum_model/checkpoint-16000/training_args.bin (deflated 49%)
  adding: content/Thaisum_model/checkpoint-16000/pytorch_model.bin (deflated 21%)
  adding: content/Thaisum_model/checkpoint-16000/special_tokens_map.json (deflated 35%)
  adding: content/Thaisum_model/checkpoint-16000/scheduler.pt (deflated 48%)
  adding: content/Thaisum_model/checkpoint-16000/rng_state.pth (deflated 28%)
  adding: content/Thaisum_model/checkpoint-16000/gene